In [ ]:
from torch.utils.data import Dataset
from PIL import Image
import os

In [ ]:
class MyData(Dataset):

    def __init__(self, root_dir,label_dir):
        self.root_dir = root_dir
        self.label_dir = label_dir
        self.path=os.path.join(self.root_dir,self.label_dir)
        self.image_path=os.listdir(self.path)

    def __getitem__(self, idx):
        image_name=self.image_path[idx]
        image=Image.open(os.path.join(self.path,image_name))
        label=self.label_dir
        return image, label

    def __len__(self):
        return len(self.image_path)

In [ ]:
root_dir='data/hymenoptera_data/train'
ants_label_dir='ants'
bees_label_dir='bees'
ants_dataset=MyData(root_dir,ants_label_dir)
bees_dataset=MyData(root_dir,bees_label_dir)
all_dataset=ants_dataset + bees_dataset